# Chapter 4: Scientific Computing with CuPy

<img src="images/cupy_title.png" style="width:600px;"/>

CuPy is a NumPy and SciPy-compatible array library for GPU-accelerated computing with Python. CuPy acts as a drop-in replacement to run existing NumPy and SciPy code on NVIDIA CUDA or AMD ROCm platforms.

CuPy is part of the Chainer project but has maintainers from many organisations including NVIDIA. CuPy implements the familiar Numpy API but with the backend written in CUDA C++. This allows folks who are already familiar with Numpy to get GPU acceleration out of the box quickly by just switching out an import.

## Links to Handy References
CuPy User Guide for more information: https://docs.cupy.dev/en/stable/user_guide/index.html 

CuPy API Reference: https://docs.cupy.dev/en/stable/reference/index.html 

CuPy Github Repository (includes more examples): https://github.com/cupy/cupy 

NumPy User Guide: https://numpy.org/doc/stable/user/ 

NumPy API Guide: https://numpy.org/doc/stable/reference/index.html 


# Examples

## A Simple Conversion from NumPy to CuPy

In [ ]:
import numpy as np
x_cpu = np.ones((1000,500,500))

x_cpu

In [ ]:
import cupy as cp
x_gpu = cp.ones((1000,500,500))
x_cpu = cp.asnumpy(x_gpu)

x_cpu

## A More Complicated Conversion from NumPy to CuPy

In [ ]:
import numpy as np

x_cpu = np.random.random((1000, 1000))
x_cpu *= 2 
u, s, v = np.linalg.svd(x_cpu)

u, s, v

In [ ]:
import cupy as cp

x_gpu = cp.random.random((1000, 1000))
x_gpu *= 2 
u, s, v = cp.linalg.svd(x_gpu)

u, s, v

## Adding a User-defined Kernel Function

In [ ]:
import cupy
from cupyx import jit


@jit.rawkernel()
def elementwise_copy(x, y, size):
    tid = jit.blockIdx.x * jit.blockDim.x + jit.threadIdx.x
    ntid = jit.gridDim.x * jit.blockDim.x
    for i in range(tid, size, ntid):
        y[i] = x[i]


size = cupy.uint32(2 ** 22)
x = cupy.random.normal(size=(size,), dtype=cupy.float32)
y = cupy.empty((size,), dtype=cupy.float32)

elementwise_copy((128,), (1024,), (x, y, size))

elementwise_copy[128, 1024](x, y, size)

assert (x == y).all()